# Analisis De Textos

1. De que tratan las empresas que se estan consitituyendo? NLP en descripcion de actividades.
2. Hay algun patron geografico? (x ej con https://kepler.gl/)
3. Hay cambios a lo largo del tiempo? Algún peak interesante?
4. Qué características de empresa tienden a pedir cuenta corriente y cuáles no?
5. Cuánto tiempo pasa entre que una empresa se constituye y solicitan una cuenta corriente?
6. Podemos ver alguna red de RUTs? Gente que esta relacionada en varias empresas (quizas con https://www.graphext.com/)


https://maartengr.github.io/BERTopic/index.html

In [1]:
#!pip install awswrangler
#!pip install --upgrade seaborn
#!pip install wordcloud
#!pip install --upgrade pandas
#!pip install unidecode
#!pip install gensim

In [32]:
%load_ext autoreload
%autoreload 2

In [31]:
import boto3
session = boto3.session.Session(profile_name='legalbot')
s3 = session.resource('s3')

In [6]:
import pandas as pd
import awswrangler as wr
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_context('poster')

from matplotlib import rcParams
rcParams['figure.figsize'] = 15,8

In [33]:
import sys
sys.path.insert(0, '../src/')

In [36]:
from features.build_features import remove_stop_phrases

In [7]:
bucket='legalres-storage-study-json'
prefix = 'processed'
data_location = 's3://{}/{}/'.format(bucket, prefix)

In [8]:
df = wr.s3.read_parquet(path=data_location,
                        boto3_session=session)

In [9]:
df.shape

(798060, 7)

In [10]:
df.columns

Index(['rut', 'cve', 'fecha', 'lugar', 'tipo_sociedad', 'objeto', 'firmas'], dtype='object')

In [11]:
df.head()

,rut,cve,fecha,lugar,tipo_sociedad,objeto,firmas
0,76304233-2,AC1DrSkE1pdl,2013-10-08,"En TEMUCO, Región de LA ARAUCANIA, Chile, a 08...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
1,76304089-5,AC1Dv0J0V4Zr,2013-10-07,"En CONCEPCION, Región del BIOBIO, Chile, a 07 ...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
2,76313238-2,AC1G6RzRn16K,2013-10-24,"En MOLINA, Región del MAULE, Chile, a 24 de oc...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
3,76313389-3,AC1G8rKdG7Xu,2013-10-24,"En COPIAPO, Región de ATACAMA, Chile, a 24 de ...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
4,76313805-4,AC1H6Pt61S4o,2013-10-29,"En TALCA, Región del MAULE, Chile, a 29 de oct...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."


## Analisis de Texto 

### Limpieza 

In [51]:
# Load the regular expression library
import re
# Remove punctuation
df['objeto_processed'] = \
df['objeto'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df['objeto_processed'] = \
df['objeto_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
df['objeto_processed'].head()

import unidecode

df['objeto_processed'] = \
df['objeto_processed'].map(lambda x: unidecode.unidecode(x))

In [54]:
df['objeto_processed'] = remove_stop_phrases(df['objeto_processed'].to_list())

In [55]:
df.loc[df['objeto_processed'].str.contains('objeto'), 'objeto_processed'].head(10)

22     a) compraventa consignacion permuta distribuci...
50     el la servicios de reserva de espacios de aten...
59     efectuar asesorias y formulacion de proyectos ...
72     a) la servicios serigraficos publicidad y otro...
75     a) la confeccion y elaboracion de proyectos y ...
101    a) la investigacion estudio asesoria capacitac...
103    la servicios de ingenieria prevencion de riesg...
122    construccion de obras y todos los actos tendie...
123    uno- arriendo de vehiculos con y sin chofer tr...
126    empresa de asesorias y servicios gastronomicos...
Name: objeto_processed, dtype: object

In [56]:
from bertopic import BERTopic

In [57]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('dccuchile/bert-base-spanish-wwm-uncased')




In [58]:
topic_model = BERTopic(embedding_model=model, language='spanish',
                       n_gram_range=(1, 3), verbose=True)

In [ ]:
topics, probs = topic_model.fit_transform(df['objeto_processed'].sample(30000).to_list())

In [ ]:
topic_model.visualize_topics(top_n_topics=30)